In [9]:
import requests
import psycopg2
import pandas as pd
import time

In [12]:

API_KEY = "cdc0473de2fb45aaa7d66e7eff2c6033"
BASE_URL = "https://newsapi.org/v2/top-headlines"


countries = ["us", "gb", "in", "ca", "au"] 
categories = ["business", "entertainment", "general", "health", "science", "sports", "technology"]

# PostgreSQL Database Connection
conn = psycopg2.connect(
    dbname="news_data",
    user="postgres",     
    password="1234", 
    host="localhost",
    port="5432"
)
cursor = conn.cursor()

cursor.execute("""
    CREATE TABLE IF NOT EXISTS news_articles (
        id SERIAL PRIMARY KEY,
        country TEXT,
        category TEXT,
        source TEXT,
        author TEXT,
        title TEXT,
        description TEXT,
        url TEXT,
        published_at TIMESTAMP
    );
""")
conn.commit()

# Loop Through Countries & Categories
for country in countries:
    for category in categories:
        print(f"Fetching news for Country: {country}, Category: {category}")

        params = {
            "country": country,
            "category": category,
            "apiKey": API_KEY
        }

        response = requests.get(BASE_URL, params=params)
        data = response.json()

        if data.get("status") == "ok": 
            articles = data.get("articles", [])

            df = pd.DataFrame(articles)
            if not df.empty:
                df = df[['source', 'author', 'title', 'description', 'url', 'publishedAt']]
                df["source"] = df["source"].apply(lambda x: x["name"] if isinstance(x, dict) else x)
                df.rename(columns={"publishedAt": "published_at"}, inplace=True)

                # Insert Data into PostgreSQL
                for _, row in df.iterrows():
                    cursor.execute("""
                        INSERT INTO news_articles (country, category, source, author, title, description, url, published_at)
                        VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
                    """, (country, category, row["source"], row["author"], row["title"], row["description"], row["url"], row["published_at"]))

                conn.commit()
                print(f"Inserted {len(df)} articles for {country} - {category}")
            else:
                print(f"No data found for {country} - {category}")

        else:
            print(f"Failed to fetch data for {country} - {category}: {data.get('message')}")

        time.sleep(2)  

# Close Database Connection
cursor.close()
conn.close()
print("ETL Process Completed!")

Fetching news for Country: us, Category: business
Inserted 19 articles for us - business
Fetching news for Country: us, Category: entertainment
Inserted 17 articles for us - entertainment
Fetching news for Country: us, Category: general
Inserted 19 articles for us - general
Fetching news for Country: us, Category: health
Inserted 17 articles for us - health
Fetching news for Country: us, Category: science
Inserted 19 articles for us - science
Fetching news for Country: us, Category: sports
Inserted 20 articles for us - sports
Fetching news for Country: us, Category: technology
Inserted 20 articles for us - technology
Fetching news for Country: gb, Category: business
No data found for gb - business
Fetching news for Country: gb, Category: entertainment
No data found for gb - entertainment
Fetching news for Country: gb, Category: general
No data found for gb - general
Fetching news for Country: gb, Category: health
No data found for gb - health
Fetching news for Country: gb, Category: sc

In [4]:
# This was to check if it really connected to Database
import psycopg2

conn = psycopg2.connect(
    dbname="news_data",
    user="postgres",
    password="1234",
    host="localhost",
    port="5432"
)

cursor = conn.cursor()
cursor.execute("SELECT current_database();")
print("Connected to Database:", cursor.fetchone()[0])  # Print the database name

cursor.close()
conn.close()


Connected to Database: news_data
